## Импорт зависимостей

In [1]:
import numpy as np
import pandas as pd

## Вспомогательные функции

In [2]:
def get_hex(number: float | int):
    if type(number) == int:
        return f'{number:x}'
    point_position: int = str(number).find(".")
    return f'{int(str(number)[0:point_position]):x}.{int(str(number)[point_position + 1:]):x}'

## Инициализация целевой функции

In [3]:
# Определение целевой функции
class TargetFunction:
    def __init__(self, name: str, middle_name: str, surname: str):
        self.i = len(name)
        self.f: float = len(middle_name)
        self.o: float = len(surname)
        self.calculations_count: int = 0
        self.derivative_calculations_count: int = 0
        self.second_derivative_calculations_count: int = 0

    def calculate(self, x: np.array, add_calculations_count: bool = True) -> float:
        if add_calculations_count:
            self.calculations_count += 1
        return self.i * x[0] ** 2 + self.o * x[1] ** 2 + self.f * x[2] ** 2 + (self.i + 10) * x[3] ** 2 + self.i * x[0] + self.o * x[1] + self.f * x[2] + 2 * x[3]

    def get_calculations_count(self) -> int:
        return self.calculations_count

    def flush_calculations_count(self) -> str:
        self.calculations_count = 0
        return 'Calculations count flush - Success'

    def calculate_partial_derivative(self, x: np.array, variable_index: int, add_calculations_count: bool = True) -> float:
        if add_calculations_count:
            self.derivative_calculations_count += 1
        if variable_index == 0:
            return 2 * self.i * x[0] + self.i
        elif variable_index == 1:
            return 2 * self.o * x[1] + self.o
        elif variable_index == 2:
            return 2 * self.f * x[2] + self.f
        elif variable_index == 3:
            return 2 * (self.i + 10) * x[3] + 2
        raise ValueError('Wrong partial')

    def get_derivative_calculations_count(self) -> int:
        return self.derivative_calculations_count

    def flush_derivative_calculations_count(self) -> str:
        self.derivative_calculations_count = 0
        return 'Derivative calculations count flush - Success'

    def calculate_partial_second_derivative(self, x: np.array, variable_index: int, add_calculations_count: bool = True) -> float:
        if add_calculations_count:
            self.second_derivative_calculations_count += 1
        if variable_index == 0:
            return 2 * self.i
        elif variable_index == 1:
            return 2 * self.o
        elif variable_index == 2:
            return 2 * self.f
        elif variable_index == 3:
            return 2 * (self.i + 10)
        raise ValueError('Wrong partial')
        
    def get_second_derivative_calculations_count(self) -> int:
        return self.second_derivative_calculations_count

    def flush_second_derivative_calculations_count(self) -> str:
        self.second_derivative_calculations_count = 0
        return 'Second derivative calculations count flush - Success'

In [4]:
# Создаем экземпляр функции
my_func: TargetFunction = TargetFunction(name='Григорий', middle_name='Григорьевич', surname='Мацнев')

## Поиск экстремума методом покоординатного градиентного спуска

In [6]:
def make_newton_step(func: TargetFunction, x_0: np.array, current_variable_index: int) -> float:
    x_1 = x_0.copy()
    # print('df', func.calculate_partial_derivative(x_0, current_variable_index))
    # print('d2f', func.calculate_partial_second_derivative(x_0, current_variable_index))
    x_1[current_variable_index] -= func.calculate_partial_derivative(x_0, current_variable_index) / func.calculate_partial_second_derivative(x_0, current_variable_index)
    return x_1


def newton_variable_descent(func: TargetFunction, x_1: np.array, precision: float = 0.001, max_iterations: int = 1000, history: pd.DataFrame = pd.DataFrame(columns=['Текущая координата', 'x_0', 'x_1', '||x_0 - x_1||', 'f(x_1)', 'Вычисления f(x)', 'Вычисления производной', 'Вычисления второй производной'])) -> (np.array, float, pd.DataFrame):
    # Инициализируем начальные значения
    indexes_dict: dict = {0: 1, 1: 2, 2: 3, 3: 0} 
    current_variable_index: int = 0
    prev_index_x: np.array = x_1
    
    # Выполняем шаги метода, пока не достигнем нужной точности
    for iteration in range(1, max_iterations + 1):
        x_0: np.array = x_1
        x_1: np.array = make_newton_step(func, x_0, current_variable_index)  
        
        history.loc[len(history)] = (current_variable_index, x_0, x_1, np.linalg.norm(x_0 - x_1), func.calculate(x_1, add_calculations_count=False), func.get_calculations_count(), func.get_derivative_calculations_count(), func.get_second_derivative_calculations_count())
        
        if np.linalg.norm(x_0 - x_1) < precision:
            if np.linalg.norm(prev_index_x - x_1) < precision:
                break
            prev_index_x = x_1
            current_variable_index = indexes_dict[current_variable_index]
            
    # Возвращаем полученные значения
    return x_1, func.calculate(x_1, add_calculations_count=False), history

In [7]:
min_x, min_value, minimizer_history = newton_variable_descent(my_func, np.array([10.0, -73.0, 65.0, 54.0]))

print(f'                  (Точка минимума, Минимальное значение) : ({min_x}, {min_value}) \n                   Количество вычислений целевой функции : {my_func.get_calculations_count()} \n       Количество вычислений производной целевой функции : {my_func.get_derivative_calculations_count()} \nКоличество вычислений второй производной целевой функции : {my_func.get_second_derivative_calculations_count()}')

_: str = my_func.flush_calculations_count()
_: str = my_func.flush_derivative_calculations_count()
_: str = my_func.flush_second_derivative_calculations_count()

minimizer_history

                  (Точка минимума, Минимальное значение) : ([-0.5        -0.5        -0.5        -0.05555556], -6.305555555555555) 
                   Количество вычислений целевой функции : 0 
       Количество вычислений производной целевой функции : 9 
Количество вычислений второй производной целевой функции : 9


,Текущая координата,x_0,x_1,||x_0 - x_1||,f(x_1),Вычисления f(x),Вычисления производной,Вычисления второй производной
0,0,"[10.0, -73.0, 65.0, 54.0]","[-0.5, -73.0, 65.0, 54.0]",1.050000e+01,131320.000000,0,1,1
1,0,"[-0.5, -73.0, 65.0, 54.0]","[-0.5, -73.0, 65.0, 54.0]",0.000000e+00,131320.000000,0,2,2
2,1,"[-0.5, -73.0, 65.0, 54.0]","[-0.5, -0.5, 65.0, 54.0]",7.250000e+01,99782.500000,0,3,3
3,1,"[-0.5, -0.5, 65.0, 54.0]","[-0.5, -0.5, 65.0, 54.0]",0.000000e+00,99782.500000,0,4,4
4,2,"[-0.5, -0.5, 65.0, 54.0]","[-0.5, -0.5, -0.5, 54.0]",6.550000e+01,52589.750000,0,5,5
5,2,"[-0.5, -0.5, -0.5, 54.0]","[-0.5, -0.5, -0.5, 54.0]",0.000000e+00,52589.750000,0,6,6
6,3,"[-0.5, -0.5, -0.5, 54.0]","[-0.5, -0.5, -0.5, -0.055555555555557135]",5.405556e+01,-6.305556,0,7,7
7,3,"[-0.5, -0.5, -0.5, -0.055555555555557135]","[-0.5, -0.5, -0.5, -0.05555555555555555]",1.582068e-15,-6.305556,0,8,8
8,0,"[-0.5, -0.5, -0.5, -0.05555555555555555]","[-0.5, -0.5, -0.5, -0.05555555555555555]",0.000000e+00,-6.305556,0,9,9


gradient_descent

In [9]:
def gradient_descent(func: TargetFunction, x_1: np.array, precision: float = 0.001, max_iterations: int = 1000, history: pd.DataFrame = pd.DataFrame(columns=['x_0', 'x_1', '||x_0 - x_1||', 'f(x_1)', 'Вычисления f(x)', 'Вычисления производной', 'Вычисления второй производной'])) -> (np.array, float, pd.DataFrame):
    step_size: float = 0.01
    
    # Выполняем шаги метода, пока не достигнем нужной точности
    for iteration in range(1, max_iterations + 1):
        x_0: np.array = x_1
        
        gradient = np.array([
            func.calculate_partial_derivative(x_0, 0),
            func.calculate_partial_derivative(x_0, 1),
            func.calculate_partial_derivative(x_0, 2),
            func.calculate_partial_derivative(x_0, 3),
        ]) 
        
        x_1 =  x_0  - step_size * gradient
        
        history.loc[len(history)] = (x_0, x_1, np.linalg.norm(x_0 - x_1), func.calculate(x_1, add_calculations_count=False), func.get_calculations_count(), func.get_derivative_calculations_count(), func.get_second_derivative_calculations_count())
        
        if np.linalg.norm(gradient) < precision:
            return x_1, func.calculate(x_1, add_calculations_count=False), history
            
    # Возвращаем полученные значения
    return x_1, func.calculate(x_1, add_calculations_count=False), history

In [10]:
min_x, min_value, minimizer_history = gradient_descent(my_func, np.array([10.0, -73.0, 65.0, 54.0]))

print(f'                  (Точка минимума, Минимальное значение) : ({min_x}, {min_value}) \n                   Количество вычислений целевой функции : {my_func.get_calculations_count()} \n       Количество вычислений производной целевой функции : {my_func.get_derivative_calculations_count()} \nКоличество вычислений второй производной целевой функции : {my_func.get_second_derivative_calculations_count()}')

_: str = my_func.flush_calculations_count()
_: str = my_func.flush_derivative_calculations_count()
_: str = my_func.flush_second_derivative_calculations_count()

minimizer_history

                  (Точка минимума, Минимальное значение) : ([-0.49999993 -0.50007319 -0.5        -0.05555556], -6.305555523412661) 
                   Количество вычислений целевой функции : 0 
       Количество вычислений производной целевой функции : 432 
Количество вычислений второй производной целевой функции : 0


,x_0,x_1,||x_0 - x_1||,f(x_1),Вычисления f(x),Вычисления производной,Вычисления второй производной
0,"[10.0, -73.0, 65.0, 54.0]","[8.32, -64.3, 50.59, 34.54]",25.784726,75294.087100,0,4,0
1,"[8.32, -64.3, 50.59, 34.54]","[6.9088, -56.644, 39.3502, 22.0856]",18.494621,45638.286088,0,8,0
2,"[6.9088, -56.644, 39.3502, 22.0856]","[5.7233920000000005, -49.90672, 30.58315600000...",13.681780,29191.845068,0,12,0
3,"[5.7233920000000005, -49.90672, 30.58315600000...","[4.7276492800000005, -43.9779136, 23.744861680...",10.436858,19500.688654,0,16,0
4,"[4.7276492800000005, -43.9779136, 23.744861680...","[3.8912253952, -38.760563968, 18.4109921104000...",8.187168,13471.454943,0,20,0
...,...,...,...,...,...,...,...
103,"[-0.49999983329226017, -0.5001386925834913, -0...","[-0.49999985996549856, -0.5001220494734724, -0...",0.000017,-6.305555,0,416,0
104,"[-0.49999985996549856, -0.5001220494734724, -0...","[-0.4999998823710188, -0.5001074035366557, -0....",0.000015,-6.305555,0,420,0
105,"[-0.4999998823710188, -0.5001074035366557, -0....","[-0.49999990119165577, -0.500094515112257, -0....",0.000013,-6.305556,0,424,0
106,"[-0.49999990119165577, -0.500094515112257, -0....","[-0.49999991700099083, -0.5000831732987862, -0...",0.000011,-6.305556,0,428,0
